In [3]:
import os

def count_files_in_folder(folder_path):
    try:
        files = os.listdir(folder_path)

        file_count = 0

        for file in files:
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                file_count += 1

        print(f"폴더 '{folder_path}' 안에 {file_count}개의 파일이 있습니다.")

    except FileNotFoundError:
        print(f"폴더 '{folder_path}' not defined")

# Data reading 

In [4]:
import json

file = json.load(open('data/textinthewild_data_info.json'))
display(file.keys(), file['info'] , type(file['images']))

dict_keys(['info', 'images', 'annotations', 'licenses'])

{'name': 'Text in the wild Dataset', 'date_created': '2019-10-14 04:31:48'}

list

In [5]:
import os 

#ocr_book_files = os.listdir('data/01_textinthewild_book_images_new/book')
ocr_good_files = os.listdir('data/01_textinthewild_goods_images_new/Goods')
#ocr_sign_files = os.listdir('data/01_textinthewild_signboard_images_new/Signboard')
#ocr_traf_files = os.listdir('data/01_textinthewild_traffic_sign_images_new/traffic_sign')

In [6]:
# ocr_files - 사진 파일 이름 저장  
ocr_files =  ocr_good_files #+ ocr_sign_files  + ocr_traf_files 

In [7]:
import random 

random.shuffle(ocr_files)

n_train = int(len(ocr_files) * 0.7) 
n_validation = int(len(ocr_files) * 0.15) 
n_test = int(len(ocr_files) * 0.15)

print(n_train, n_validation, n_test)  

train_files = ocr_files[:n_train]
validation_files = ocr_files[n_train: n_train+n_validation]
test_files = ocr_files[-n_test:]

18440 3951 3951


In [8]:
## train/validation/test 이미지들에 해당하는 id 값을 저장
# 이때 id 값은 json 파일에서 가져온 것임 
# 해당 id 값들을 train,test,val 에 맞게 spliting 한 것임 

train_img_ids = {}
validation_img_ids = {}
test_img_ids = {}

for image in file['images']:
    if image['file_name'] in train_files:
        train_img_ids[image['file_name']] = image['id']
        
    elif image['file_name'] in validation_files:
        validation_img_ids[image['file_name']] = image['id']
        
    elif image['file_name'] in test_files:
        test_img_ids[image['file_name']] = image['id']

In [9]:
display(len(train_img_ids),len(validation_img_ids),len(test_img_ids))

18440

3951

3951

In [10]:
## train/validation/test 이미지들에 해당하는 annotation 들을 저장

train_annotations = {f:[] for f in train_img_ids.keys()} #keys -> 파일 이름  # 파일이름 : [] 로 value 가 비어있는 딕셔내리 형성 
validation_annotations = {f:[] for f in validation_img_ids.keys()}
test_annotations = {f:[] for f in test_img_ids.keys()}

train_ids_img = {train_img_ids[id_]:id_ for id_ in train_img_ids}  # id : 해당 파일 이름 -> 으로 구성된 딕셔내리 
validation_ids_img = {validation_img_ids[id_]:id_ for id_ in validation_img_ids} 
test_ids_img = {test_img_ids[id_]:id_ for id_ in test_img_ids}

for idx, annotation in enumerate(file['annotations']):
    if idx % 5000 == 0:
        print(idx,'/',len(file['annotations']),'processed')
    if annotation['attributes']['class'] != 'word': # class 가 word 가 아니면, 아래 반복문 실행 X 
        continue
    if annotation['image_id'] in train_ids_img:
        train_annotations[train_ids_img[annotation['image_id']]].append(annotation)
    elif annotation['image_id'] in validation_ids_img:
        validation_annotations[validation_ids_img[annotation['image_id']]].append(annotation)
    elif annotation['image_id'] in test_ids_img:
        test_annotations[test_ids_img[annotation['image_id']]].append(annotation)

with open('train_annotation.json', 'w') as file:
    json.dump(train_annotations, file)
with open('validation_annotation.json', 'w') as file:
    json.dump(validation_annotations, file)
with open('test_annotation.json', 'w') as file:
    json.dump(test_annotations, file)

0 / 2096460 processed
5000 / 2096460 processed
10000 / 2096460 processed
15000 / 2096460 processed
20000 / 2096460 processed
25000 / 2096460 processed
30000 / 2096460 processed
35000 / 2096460 processed
40000 / 2096460 processed
45000 / 2096460 processed
50000 / 2096460 processed
55000 / 2096460 processed
60000 / 2096460 processed
65000 / 2096460 processed
70000 / 2096460 processed
75000 / 2096460 processed
80000 / 2096460 processed
85000 / 2096460 processed
90000 / 2096460 processed
95000 / 2096460 processed
100000 / 2096460 processed
105000 / 2096460 processed
110000 / 2096460 processed
115000 / 2096460 processed
120000 / 2096460 processed
125000 / 2096460 processed
130000 / 2096460 processed
135000 / 2096460 processed
140000 / 2096460 processed
145000 / 2096460 processed
150000 / 2096460 processed
155000 / 2096460 processed
160000 / 2096460 processed
165000 / 2096460 processed
170000 / 2096460 processed
175000 / 2096460 processed
180000 / 2096460 processed
185000 / 2096460 processed

In [11]:
import cv2
from tqdm import tqdm

In [12]:
## aihub 데이터 annotation을 읽어서 단어 단위로 잘라서 data에 저장하기

data_root_path = 'data/ocr_solut/'
save_root_path = 'deep-text-recognition-benchmark/data/'

# test 
test_annotations = json.load(open('./test_annotation.json'))
gt_file = open(save_root_path+'gt_test.txt', 'w')


for file_name in tqdm(test_annotations):
    annotations = test_annotations[file_name]
    image = cv2.imread(data_root_path+file_name)
    for idx, annotation in enumerate(annotations):
        x,y,w,h = annotation['bbox']
        if x<= 0 or y<= 0 or w <= 0 or h <= 0:
            continue
        text = annotation['text']
        crop_img = image[y:y+h,x:x+w]
        crop_file_name = file_name[:-4]+'_{:03}.jpg'.format(idx+1)
        cv2.imwrite(save_root_path+'test/'+crop_file_name, crop_img)
        gt_file.write("test/{}\t{}\n".format(crop_file_name, text))

# Close the file after writing
gt_file.close()

100%|███████████████████████████████████████| 3951/3951 [02:52<00:00, 22.86it/s]


In [13]:
# img 수 
count_files_in_folder(save_root_path+'test')

폴더 'deep-text-recognition-benchmark/data/test' 안에 11810개의 파일이 있습니다.


In [14]:
#txt file 길이 
with open(save_root_path+'gt_test.txt', 'r') as file:
    lines = file.readlines()
    num_lines = len(lines)
    #print(content)
    print(f'The file contains {num_lines} lines.')

The file contains 11810 lines.


In [16]:
# validation 
validation_annotations = json.load(open('./validation_annotation.json'))
gt_file = open(save_root_path+'gt_validation.txt', 'w')

for file_name in tqdm(validation_annotations):
    annotations = validation_annotations[file_name]
    image = cv2.imread(data_root_path+file_name)
    for idx, annotation in enumerate(annotations):
        x,y,w,h = annotation['bbox']
        if x<= 0 or y<= 0 or w <= 0 or h <= 0:
            continue        
        text = annotation['text']
        crop_img = image[y:y+h,x:x+w]
        crop_file_name = file_name[:-4]+'_{:03}.jpg'.format(idx+1)
        cv2.imwrite(save_root_path+'validation/'+crop_file_name, crop_img)
        gt_file.write("validation/{}\t{}\n".format(crop_file_name, text))

# Close the file after writing
gt_file.close()

100%|███████████████████████████████████████| 3951/3951 [03:01<00:00, 21.74it/s]


In [17]:
# img 수 
count_files_in_folder(save_root_path+'validation')

폴더 'deep-text-recognition-benchmark/data/validation' 안에 11929개의 파일이 있습니다.


In [18]:
#txt file 길이 
with open(save_root_path+'gt_validation.txt', 'r') as file:
    lines = file.readlines()
    num_lines = len(lines)
    #print(content)
    print(f'The file contains {num_lines} lines.')

The file contains 11929 lines.


In [20]:
# train 
train_annotations = json.load(open('./train_annotation.json'))
gt_file = open(save_root_path+'gt_train.txt', 'w')


for file_name in tqdm(train_annotations):
    annotations = train_annotations[file_name]
    image = cv2.imread(data_root_path + file_name)

    if image is None or image.size == 0:
        # Skip processing empty images
        continue

    for idx, annotation in enumerate(annotations):
        x, y, w, h = annotation['bbox']
        if x <= 0 or y <= 0 or w <= 0 or h <= 0:
            continue

        text = annotation['text']
        crop_img = image[y:y + h, x:x + w]

        if crop_img.size == 0:
            # Skip processing empty cropped images
            continue

        crop_file_name = file_name[:-4] + '_{:03}.jpg'.format(idx + 1)
        cv2.imwrite(save_root_path + 'train/' + crop_file_name, crop_img)
        gt_file.write("train/{}\t{}\n".format(crop_file_name, text))

gt_file.close()

100%|█████████████████████████████████████| 18440/18440 [14:03<00:00, 21.86it/s]


In [21]:
# img 수 
count_files_in_folder(save_root_path+'train')

폴더 'deep-text-recognition-benchmark/data/train' 안에 55443개의 파일이 있습니다.


In [23]:
#txt file 길이 
with open(save_root_path+'gt_train.txt', 'r') as file:
    lines = file.readlines()
    num_lines = len(lines)
    #print(content)
    print(f'The file contains {num_lines} lines.')

The file contains 55443 lines.


In [25]:
#validation -> val 

file_path = 'deep-text-recognition-benchmark/data/gt_validation.txt'

with open(file_path, 'r') as file:
    content = file.read()

# "validation"을 "val"로 바꾸기
new_content = content.replace('validation', 'val')

with open(file_path, 'w') as file:
    file.write(new_content)